#### Setup access token

In [1]:
import pandas as pd
import webbrowser
# Import the required module from the fyers_apiv3 package
from fyers_apiv3 import fyersModel
from datetime import datetime, timedelta
import pandas as pd
import time

# Configuration

In [2]:
def generate_fyers_access_token(
    client_id: str,
    secret_key: str,
    redirect_uri: str,
    state: str = "sample_state"
) -> dict:
    """
    Handles FYERS OAuth flow and returns access token response.

    Returns
    -------
    dict
        Full response containing access_token, refresh_token, expiry, etc.
    """

    # -------------------------------
    # Step 1: Generate auth URL
    # -------------------------------
    session = fyersModel.SessionModel(
        client_id=client_id,
        secret_key=secret_key,
        redirect_uri=redirect_uri,
        response_type="code",
        state=state
    )

    auth_url = session.generate_authcode()
    webbrowser.open(auth_url)

    # -------------------------------
    # Step 2: User pastes redirected URL
    # -------------------------------
    raw_code = input("Paste redirected URL here: ").strip()

    try:
        auth_code = raw_code.split("auth_code=")[1].split("&")[0]
    except IndexError:
        raise ValueError("Invalid redirect URL. Auth code not found.")

    # -------------------------------
    # Step 3: Exchange auth_code for access_token
    # -------------------------------
    session = fyersModel.SessionModel(
        client_id=client_id,
        secret_key=secret_key,
        redirect_uri=redirect_uri,
        response_type="code",
        grant_type="authorization_code"
    )

    session.set_token(auth_code)
    token_response = session.generate_token()

    return token_response


In [3]:
client_id="ALT6RUE1IF-100"
secret_key="0UT0LW5PE4"
redirect_uri="https://luvratan.tech/"

token_response = generate_fyers_access_token(
    client_id=client_id,
    secret_key=secret_key,
    redirect_uri=redirect_uri
)

access_token = token_response["access_token"]
print("Access Token:", access_token)


Access Token: eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhdWQiOlsiZDoxIiwiZDoyIiwieDowIiwieDoxIl0sImF0X2hhc2giOiJnQUFBQUFCcGdHTVNfWElacGZQcDR2RXpyR3dnRldCTDFqeFlIZlI0RS01YkNCUEFEcEFueEZhY05NOVB6NlZ4UmxwOFBNOE5IVWJKNVJRT19MQlVEdy1LcllhQ0hyVjRsYWFZdXN3WmJuS0EyRWl0eU9EVFAxST0iLCJkaXNwbGF5X25hbWUiOiIiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiJiMjM2ZDk2Mjg3YzdkYjI0NzdmOTE0Y2FhZWQ5Y2YwZTNmYTAxYzQxMzgzMWIxMmMxYjRhYjlhYyIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImZ5X2lkIjoiWEwwMDcyMSIsImFwcFR5cGUiOjEwMCwiZXhwIjoxNzcwMDc4NjAwLCJpYXQiOjE3NzAwMjE2NTAsImlzcyI6ImFwaS5meWVycy5pbiIsIm5iZiI6MTc3MDAyMTY1MCwic3ViIjoiYWNjZXNzX3Rva2VuIn0.qipXMIldG6Hh-unwG_Ju_TOSvXHZJT-53-9TA-FtbxQ


#### Fetch Data

In [14]:
import time

resolution = 5
bars_needed = 203

# Request 10 days of data to ensure we have enough bars
days_to_fetch = 10
lookback_seconds = days_to_fetch * 24 * 60 * 60  # 10 days in seconds

# Avoid partial candles as per API documentation
range_to = int(time.time()) - (resolution * 60)
range_from = range_to - lookback_seconds

print(f"Fetching {days_to_fetch} days of {resolution}-minute data")

Fetching 10 days of 5-minute data


In [15]:

# Initialize the FyersModel instance with your client_id, access_token, and enable async mode
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")

data = {
    "symbol": "NSE:ADANIPORTS-EQ",
    "resolution": str(resolution),
    "date_format": "0",
    "range_from": str(range_from),
    "range_to": str(range_to),
    "cont_flag": "1"
}

response = fyers.history(data=data)
print(response)

{'candles': [[1769157900, 1302.1, 1309, 1295, 1305.8, 266637], [1769158200, 1305.3, 1314.8, 1303.5, 1308, 261105], [1769158500, 1308, 1308.8, 1304, 1308.2, 133692], [1769158800, 1307.9, 1308.9, 1302.2, 1307.7, 123654], [1769159100, 1307.7, 1308.2, 1299.4, 1301.7, 141716], [1769159400, 1301.7, 1309.4, 1301.4, 1305.6, 153479], [1769159700, 1307.2, 1308.2, 1297.3, 1298.5, 138751], [1769160000, 1298, 1314.9, 1292, 1314, 283581], [1769160300, 1316.3, 1325, 1311.5, 1318.6, 336895], [1769160600, 1318.6, 1320.6, 1303.5, 1303.5, 208447], [1769160900, 1302.8, 1308.2, 1300.2, 1307.6, 188944], [1769161200, 1308.1, 1311.1, 1304.7, 1309.5, 174818], [1769161500, 1308.7, 1314, 1308, 1310.5, 185035], [1769161800, 1310.6, 1313, 1305.3, 1310.7, 157324], [1769162100, 1310.8, 1314.9, 1306.3, 1314.9, 157921], [1769485500, 1334.3, 1344, 1322.7, 1331.5, 248500], [1769485800, 1331.7, 1333.7, 1322, 1332.8, 104894], [1769486100, 1332.8, 1344.9, 1330.6, 1343.8, 107133], [1769486400, 1343.8, 1347.4, 1338.2, 1340.8

In [16]:
def fyers_history_to_df(response: dict) -> pd.DataFrame:
    """
    Convert FYERS history API response to a pandas DataFrame.
    Shifts intraday candle timestamps forward by 15 minutes.
    """

    if response.get("s") != "ok":
        raise ValueError(f"FYERS API error: {response}")
    print(response["candles"])
    df = pd.DataFrame(
        response["candles"],
        columns=["timestamp", "open", "high", "low", "close", "volume"]
    )

    # Convert UNIX epoch → timezone-aware datetime (IST)
    df["datetime"] = (
        pd.to_datetime(df["timestamp"], unit="s", utc=True)
        .dt.tz_convert("Asia/Kolkata")
    )

    # Shift candle time forward by n minutes
    # df["datetime"] = df["datetime"] + pd.Timedelta(minutes=resolution)

    # Sort by adjusted time
    df = df.sort_values("datetime")

    # Drop raw timestamp if not needed
    df = df.drop(columns=["timestamp"])
    df = df.set_index("datetime")

    return df


df = fyers_history_to_df(response)


[[1769157900, 1302.1, 1309, 1295, 1305.8, 266637], [1769158200, 1305.3, 1314.8, 1303.5, 1308, 261105], [1769158500, 1308, 1308.8, 1304, 1308.2, 133692], [1769158800, 1307.9, 1308.9, 1302.2, 1307.7, 123654], [1769159100, 1307.7, 1308.2, 1299.4, 1301.7, 141716], [1769159400, 1301.7, 1309.4, 1301.4, 1305.6, 153479], [1769159700, 1307.2, 1308.2, 1297.3, 1298.5, 138751], [1769160000, 1298, 1314.9, 1292, 1314, 283581], [1769160300, 1316.3, 1325, 1311.5, 1318.6, 336895], [1769160600, 1318.6, 1320.6, 1303.5, 1303.5, 208447], [1769160900, 1302.8, 1308.2, 1300.2, 1307.6, 188944], [1769161200, 1308.1, 1311.1, 1304.7, 1309.5, 174818], [1769161500, 1308.7, 1314, 1308, 1310.5, 185035], [1769161800, 1310.6, 1313, 1305.3, 1310.7, 157324], [1769162100, 1310.8, 1314.9, 1306.3, 1314.9, 157921], [1769485500, 1334.3, 1344, 1322.7, 1331.5, 248500], [1769485800, 1331.7, 1333.7, 1322, 1332.8, 104894], [1769486100, 1332.8, 1344.9, 1330.6, 1343.8, 107133], [1769486400, 1343.8, 1347.4, 1338.2, 1340.8, 102223], [

In [17]:
df.head()

,open,high,low,close,volume
datetime,,,,,
2026-01-23 14:15:00+05:30,1302.1,1309.0,1295.0,1305.8,266637
2026-01-23 14:20:00+05:30,1305.3,1314.8,1303.5,1308.0,261105
2026-01-23 14:25:00+05:30,1308.0,1308.8,1304.0,1308.2,133692
2026-01-23 14:30:00+05:30,1307.9,1308.9,1302.2,1307.7,123654
2026-01-23 14:35:00+05:30,1307.7,1308.2,1299.4,1301.7,141716


In [18]:
df.to_csv("demo_data.csv", index="datetime")